# Get the year in the date_created field
First find all year in dates with four digits, then three, then two.
For a date range, the function will return to all years, for example, it will return [1926, 1940] for [between 1926-11-30 and 1940-11-02]. The special charactors ("?","-") will also be extracted.

In [41]:
# Use Regular Expression Module in python to extract dates. 
# Documentation available at https://docs.python.org/3.6/library/re.html
import re

In [42]:
def getDate(s):
#This expression will return to the 
    d=re.findall(r'([1-3][0-9]{3}.*?)', s);
    if len(d)==0:
        d=re.findall(r'([1-3][0-9]{2}\D)',s);
        if len(d)==0:
            d=re.findall(r'([1-3][0-9]{1}\D\D)',s);
#Always ensure the second date is larger than the first date 
    if len(d)==2:
        if d[0]>d[1]:
            d[0],d[1]=d[1],d[0]
    return d

In [43]:
getDate("[between 196?-11-04 and 196?-11-08]")

['196?', '196?']

In [44]:
getDate("[189-?]")

['189-']

In [45]:
getDate("[between 1967-11-04 and 1965-11-08]")

['1965', '1967']

# Extract start and enddate
Get the start and end date from dates extract from the date_created field. 
* For four digits dates, it will return to the all four digit
* For three digits dates, it will return to the first three digits and 0 or 9, e.g. start date for 191- is 1910, end date is 1919
* For two digits dates (centry certain), it will return to the first two digits and 00 or 99, e.g. start date for 19-? is 1900, end date is 1999

For the dates only have one year (1890, [ca,1890], 1890?), the start and end date will both be set as 1890.

In [46]:
# The function to extract startDate
def startDate(d):
    if len(d)>0:
        if re.match(r'([1-3][0-9]{3}.*?)',d[0]):
            return d[0][:4];
        if re.match(r'([1-3][0-9]{2}\D)',d[0]):
            return d[0][:3]+'0';
        if re.match(r'([1-3][0-9]{1}\D\D)',d[0]):
            return d[0][:2]+'00'

In [47]:
startDate(['19--'])

'1900'

In [48]:
startDate(getDate("[between 196?-11-04 and 196?-11-08]"))

'1960'

In [49]:
#The function to extract endDate
def endDate(d):
    if len(d)==2:
        if re.match(r'([1-3][0-9]{3}.*?)',d[1]):
            return d[1][:4];
        elif re.match(r'([1-3][0-9]{2}\D)',d[1]):
            return d[1][:3]+'9';
        elif re.match(r'([1-3][0-9]{1}\D\D)',d[1]):
            return d[1][:2]+'99';
    if len(d)==1:
        if re.match(r'([1-3][0-9]{3}.*?)',d[0]):
            return d[0][:4];
        elif re.match(r'([1-3][0-9]{2}\D)',d[0]):
            return d[0][:3]+'9';
        elif re.match(r'([1-3][0-9]{1}\D\D)',d[0]):
            return d[0][:2]+'99'; 

In [50]:
endDate(['193?'])

'1939'

In [51]:
endDate(['19--'])

'1999'

In [52]:
endDate(getDate("[between 196?-11-04 and 196?-11-08]"))

'1969'

# Add Start and End Dates for records

In [53]:
# Use pandas to store the data
# Documentation of pandas available at http://pandas.pydata.org/pandas-docs/stable/
import pandas as pd

### Import the csv file that contains the ID and the date
dates.csv should look like

| ID        | RBSC_Date Created |
| ------------- |-------------:|
| 1      | [ca. 1920] |
| 2      | [1918?] 	     |
| 3 | [between 1930 and 1939?]   |


In [54]:
df_date = pd.read_csv('dates.csv')

In [55]:
df_date =df_date.set_index("ID")

In [56]:
# Add two new columns named "startDates" and "endDates"
col_names=['startDates','endDates']
df_date_new=pd.DataFrame(columns=col_names)
df_date=df_date.append(df_date_new)

C:\Users\cuican61\Programs\anoconda\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [57]:
df_date.head()

,RBSC_Date Created,endDates,startDates
1,[ca. 1920],NaN,NaN
2,[1918?],NaN,NaN
3,[between 1930 and 1939?],NaN,NaN
4,1939-05-06,NaN,NaN
5,1921-07-02,NaN,NaN


### Loop through all records to add start and end date

In [58]:
for i,v in df_date['RBSC_Date Created'].iteritems():
    if type(v)!=float:
        df_date['startDates'][i]=startDate(getDate(v));
        df_date['endDates'][i]=endDate(getDate(v))

In [59]:
df_date.head()

,RBSC_Date Created,endDates,startDates
1,[ca. 1920],1920,1920
2,[1918?],1918,1918
3,[between 1930 and 1939?],1939,1930
4,1939-05-06,1939,1939
5,1921-07-02,1921,1921


In [60]:
#Export the date to csv file
df_date.to_csv('startAndEndDate.csv')


---
## References
1. Regular Expression (re) documentation https://docs.python.org/3.6/library/re.html
2. Pandas documentation http://pandas.pydata.org/pandas-docs/stable/

---

## Appendix 1: Dates formats in Chung Metadata
#### Case 1: Dates contains four-digits year
* Between YYYY[-MM-DD] and YYYY[-MM-DD]
    * [between 1950 and 1979]
    * [between 1860 and 1939?]
    * [between 1915 and 1918]
    * [between 1926-11-30 and 1940-11-02]
    * [between 1928-06-24 and 1928-07-01?]
    * between 1910-09-20 and 1911-03-18?] 
    * between 1957-02 and 1957-03
    * [between 1922-1989?]
* YYYY
    * [1860?]
    * [1864]
    * [1885-11-07]
    * [1900-11]
    * [1996-01-13?]
    * [1838-01]
    * 1852
* YYYY-YYYY
    * [1860-1870]
    * [1870-1880?]
    * [2001]-2008
    * 1828-1829
* [DD]-MMM-YYYY
    * [24 Mar.- 11 June 1913]
    * 27 July 2001
    * May 1937
* [not after 1907]
* [not before 1800]
* [ca. 1879]

#### Case 2: Dates contains three-digits year
* [189-?], [191-]
* [between 193?-09-18 and 193?-09-26]
* [193?-07-19]

#### Case 3: Dates contains two-digits year
* 2 Aug. [19-]

#### Others
* [n.d.]
* [unknown]

## Appendix 2: test on regular expression

### Case 1: Four-digits year

In [61]:
import re

In [62]:
s="[between 1961-11-04 and 1961-11-08]"
d = re.findall(r'([1-3][0-9]{3}.*?)', s)
d

['1961', '1961']

In [63]:
s='[ca. 1879]'
d = re.findall(r'([1-3][0-9]{3}.*?)', s)
d

['1879']

In [64]:
s='[2001]-2008'
d = re.findall(r'([1-3][0-9]{3}.*?)', s)
d

['2001', '2008']

In [65]:
s='1962'
d = re.findall(r'([1-3][0-9]{3}.*?)', s)
d

['1962']

In [66]:
s='[24 Mar.- 11 June 1913]'
d = re.findall(r'([1-3][0-9]{3}.*?)', s)
d

['1913']

### Case 2: Three-digits year
including 199?, [193?-07-19], [193-], '[between 196?-11-04 and 196?-11-08]'

In [67]:
s='199?'
re.match('\d\d\d.?',s)
print(s[:3])

199


In [68]:
s='[193?-07-19]'
d=re.findall(r'([1-3][0-9]{2}.+?)',s)
d

['193?']

In [69]:
s='[between 196?-11-04 and 196?-11-08]'
d=re.findall(r'([1-3][0-9]{2}.+?)',s)
d

['196?', '196?']

In [70]:
s='[193-]'
d=re.findall(r'([1-3][0-9]{2}.+-)',s)
d

[]

In [71]:
s='[between 196?-11-04 and 196?-11-08]'
d=re.findall(r'([1-3][0-9]{2}\D)',s)
d

['196?', '196?']

In [72]:
s='[193-?]'
d=re.findall(r'([1-3][0-9]{2}\D)',s)
d

['193-']

### Case 3: Two-digits year

In [73]:
s='[19--?]'
d=re.findall(r'([1-3][0-9]{1}\D\D)',s)
d

['19--']

### Universal

In [74]:
st=['[24 Mar.- 11 June 1913]', "[between 1961-11-04 and 1961-11-08]",'[ca. 1879]',"[19--?]",'[between 196?-11-04 and 196?-11-08]']
for s in st:
    d=re.findall(r'([1-3][0-9]{1}\S\S)',s)
    print(d)

['1913']
['1961', '11-0', '1961', '11-0']
['1879']
['19--']
['196?', '11-0', '196?', '11-0']
